## Initializing

Not all 0's : weight 가 0이면 모든 gradient가 0이되어 back propagation 학습을 할 수 없다.

### RBM : restricted boltzmann machine

restriced : no connections with in a layer

다른 layer 사이에는 연결이 되어있다.

### Xavier / He initializion 

Initiallization을 무작위가 아닌 layer 특성에 따라 하는 방법

- Xavier/He Normal Initialization

- Xavier/He Uniform initialization

## Code : mnist__nn__deep

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [3]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root = 'MNIST_data/', train = True, transform = transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root = 'MNIST_data/', train = False, transform=transforms.ToTensor(),
                         download = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size,
                                          shuffle = True, drop_last = True)

In [7]:
linear1 = torch.nn.Linear(784,512, bias = True)
linear2 = torch.nn.Linear(512,512, bias = True)
linear3 = torch.nn.Linear(512,512, bias = True)
linear4 = torch.nn.Linear(512,512, bias = True)
linear5 = torch.nn.Linear(512,10, bias = True)
relu = torch.nn.ReLU()

In [9]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0609,  0.0468, -0.0650,  ...,  0.0986, -0.0188, -0.0318],
        [-0.0608,  0.0482, -0.0718,  ..., -0.0226, -0.0302,  0.0074],
        [-0.0618,  0.0946,  0.0216,  ..., -0.1030,  0.0172,  0.1070],
        ...,
        [ 0.1065, -0.1039,  0.0298,  ..., -0.0079,  0.0018,  0.0794],
        [-0.0266,  0.0046, -0.1046,  ...,  0.0276,  0.0209,  0.0399],
        [-0.0643, -0.0725,  0.0499,  ..., -0.0865, -0.0614,  0.1016]],
       requires_grad=True)

In [10]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [11]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [13]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    #reshape input image into [batch_size by 784]
    #label is not one-hot encoded

    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis,Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/ total_batch

  print('Epoch:', '%04d' %(epoch +1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished')

Epoch: 0001 cost = 0.277822077
Epoch: 0002 cost = 0.088015452
Epoch: 0003 cost = 0.056250393
Epoch: 0004 cost = 0.039453488
Epoch: 0005 cost = 0.031990781
Epoch: 0006 cost = 0.024698051
Epoch: 0007 cost = 0.019154049
Epoch: 0008 cost = 0.019998115
Epoch: 0009 cost = 0.015274491
Epoch: 0010 cost = 0.014153772
Epoch: 0011 cost = 0.013547207
Epoch: 0012 cost = 0.013091481
Epoch: 0013 cost = 0.010799323
Epoch: 0014 cost = 0.009673556
Epoch: 0015 cost = 0.010452357
Learning Finished


In [16]:
#Test the model using test sets

with torch.no_grad():
  X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction,1) ==Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy :', accuracy.item())

  #Get one and predict
  r= random.randint(0, len(mnist_test)-1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print("Label:", Y_single_data.item())
  single_prediction = model(X_single_data)
  print("Prediction:", torch.argmax(single_prediction,1).item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy : 0.9800999760627747
Label: 4
Prediction: 4
